## Gonway's game of life

In [ ]:
import pygame

from pygame.locals import *
import sys   
import random

from cell import Cell

import time
import copy

from file_game import dialogGame

pygame.init()
clock = pygame.time.Clock()
font = pygame.font.SysFont("Helvetica", 16)
font.set_bold(True)

WHITE = (255,255,255)
RED = (255,0,0)
GREEN = (0,255,0)



In [ ]:
class GameOfLive(): ## 
    def __init__(self):
        pygame.display.set_caption("Conway's game of life")

        self.clock = pygame.time.Clock()
        self.last_tick = pygame.time.get_ticks()

        self.HEIGHT = 100 ## 49
        self.WIDTH  = 120 ##  74
        self.SIDE_SQ = 10
        self.screen_res = [self.WIDTH* self.SIDE_SQ, self.HEIGHT* self.SIDE_SQ]

        self.sprites = pygame.sprite.Group()
        self.cells = []
        self.generation = 0
        self.population = 0
        
        self.screen = pygame.display.set_mode(self.screen_res, pygame.HWSURFACE, 32)

        self.createGrid()
        
        print('Grid is ready')
        
        self.time_start = 0
        self.time_game = 0
        self.startGame = False
        self.gameLoop = True
        self.compare_gen = False
        self.fps = 8
        
        self.dialogGame = dialogGame(self)
        
    def update_title(self, title):
        pygame.display.set_caption(title)
        
    def game(self):    
                
        while self.gameLoop: ## Loop
            # the main event loop, detects keypresses
            for event in pygame.event.get():
                if event.type == QUIT or (event.type == KEYDOWN and event.key == K_ESCAPE):
                    self.gameLoop = False
                    
                if event.type == KEYDOWN:
                    
                    if event.key == K_RETURN:
                        self.time_start = time.time()
                        self.startGame = True
                        
                    if event.key == K_SPACE:
                        tdelta = time.time() - self.time_start
                        self.time_game = round(tdelta, 2)
                        self.sprites.empty()
                        self.cells = []
                        self.createGrid()
                        self.time_start = 0
                        self.startGame = False
                        self.compare_gen = False
                        self.generation = 0
                        self.population = 0
                        
                    if event.key == K_r:
                        paused = True
                        while paused == True:
                            clock.tick(50)
                            pygame.event.pump()
                            for event in pygame.event.get():
                                 if event.type == KEYDOWN:
                                    paused = False          
                                    
                    if event.key == K_f:
                        self.dialogGame.readFileDialog()
                        
                    if event.key == K_w:
                        self.dialogGame.saveFileDialog()    
                        
                    if event.key == K_x:
                        fps = self.dialogGame.getValue()
                        print('X-fps: ', fps)
                        if fps != -1:
                            self.fps = fps
                    
        
            self.Tick()
            self.Draw()                   
            pygame.display.update()
            
        pygame.quit()
        sys.exit()      

    def createGrid(self):
        col = 0
        row = 50
        cell_num = 0

        ## Each cell is 10x10 square, 
        ## rows = (480 - 50) /10 = 43, cols 730/10 = 73
        ## height_cells = 44
        ## width_cells = 74
        for y in range(self.HEIGHT-5): ## (44 = 49-5):
            for x in range(self.WIDTH): ##  (74):
                cell_num +=1
                cell = Cell(self, [col, row], cell_num)
                if row == 5* self.SIDE_SQ or row  == (self.HEIGHT - 1)* self.SIDE_SQ \
                             or col == 0 or col == (self.WIDTH - 1)* self.SIDE_SQ: 
                     cell.edge = True
                self.sprites.add(cell)
                col +=  self.SIDE_SQ
            row +=  self.SIDE_SQ
            col = 0
          
    def GetNextGeneration(self):
        self.population = 0
        
        for cell in self.cells:
            
            cell.prev_alive[0] = cell.prev_alive[1]
            cell.prev_alive[1] = cell.alive ## Save previous cell.alive value
            
            if cell.alive:
                self.population += 1
                if len(cell.a_neighbors) < 2:
                    cell.die()
                elif len(cell.a_neighbors) > 3:
                    cell.die()
                elif len(cell.a_neighbors) == 2 or len(cell.a_neighbors) == 3:
                    cell.live()
            else:
                if len(cell.a_neighbors) == 3:
                    cell.live()
                    
    def blitDirections(self):
        text = font.render("Enter:start    Space:stop and clear    Esc:exit    r: pause    f: read file    w: save   x: get fps    ", 1, WHITE)        
        gentext = font.render("Gen: %s" %str(self.generation), 1, RED)
        poptext = font.render("Pop: %s" %str(self.population), 1, GREEN) 
        
        if self.time_game > 0:
            timetext = font.render("Time: %s" %str(self.time_game), 1, WHITE) 
        else:
            timetext = font.render("Time: %s" %str(0), 1, WHITE)  
            
        self.screen.blit(text, (10, 18))
        self.screen.blit(gentext, (700, 18))
        self.screen.blit(poptext, (800, 18))        
        self.screen.blit(timetext, (900, 18)) 

    def Tick(self):   
        # updates to player location and animation frame
        self.mpos = pygame.mouse.get_pos()
        self.keys_pressed = pygame.key.get_pressed()
        if self.startGame:
            if self.generation > 0:
                if self.Compare() == True: ## prev_alive[0] or [1] = alive for all cells
                    self.compare_gen = True
                    self.GetNextGeneration()
                else:
                    if self.compare_gen == False:
                        self.generation +=1
                        tdelta = time.time() - self.time_start
                        self.time_game = round(tdelta, 2)

                    self.rumGame = True
                    self.gameLoop = True
                    self.GetNextGeneration()
            else:  
                self.generation +=1
                ## print('self.generation = 0 and self.startGame = True ')
                tdelta = time.time() - self.time_start
                self.time_game = round(tdelta, 2)
                self.startGame = True
                self.gameLoop = True
                self.GetNextGeneration()
                          

    def Draw(self):
        self.screen.fill(0)
        clock.tick(self.fps) ## (8), (4)
        self.blitDirections()
        self.sprites.update()  ## Here, the sprites are setting alive/not alive
        self.sprites.draw(self.screen)
        
    def Compare(self):

        prev_0 = True
        prev_1 = True
        
        for cell in self.cells:
            if cell.prev_alive[1] != cell.alive:             
                prev_1 = False
                break
                        
        for cell in self.cells:
            if cell.prev_alive[0] != cell.alive:             
                prev_0 = False
                break
        
        if prev_0 == True or prev_1 == True:
             return True
        


In [ ]:
GameOfLive().game()